# Gridflag Statistics

This notebook shows some usage of the Xarray-MS Dataset and the dask delayed functions.  

12/15/2019:
This is meant to be a demo notebook and an example of computing statistics with Dask/Xarray/Dask-ms but it is very messy right now.  Will clean up in the coming days. 

In [1]:
import numpy as np

import dask
import dask.array as da

from compute_uv_bins import load_ms_file
import groupby_apply

In [2]:
ds_ind = load_ms_file('/scratch/users/jbochenek/data/1491550051.ms/')

In [3]:
ds_ind

<xarray.Dataset>
Dimensions:  (corr: 4, newrow: 336461824, uvw: 2)
Coordinates:
    U_bins   (newrow) int64 dask.array<chunksize=(10000000,), meta=np.ndarray>
    V_bins   (newrow) int64 dask.array<chunksize=(10000000,), meta=np.ndarray>
  * newrow   (newrow) MultiIndex
  - row      (newrow) int64 0 0 0 0 0 0 ... 82143 82143 82143 82143 82143 82143
  - chan     (newrow) int64 0 1 2 3 4 5 6 ... 4089 4090 4091 4092 4093 4094 4095
Dimensions without coordinates: corr, uvw
Data variables:
    DATA     (newrow, corr) complex64 dask.array<chunksize=(10000000, 4), meta=np.ndarray>
    UV       (newrow, uvw) float64 dask.array<chunksize=(10000000, 2), meta=np.ndarray>

---

## Plot UV Coverage

In [56]:
from bokeh.plotting import figure, show, output_file

In [57]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [8]:
uval = ds_ind.UV[:,0]
vval = ds_ind.UV[:,1]

In [ ]:
uval = ds_vis.uvw_scaled[:,:,0]
vval = ds_vis.uvw_scaled[:,:,1]

In [9]:
# Take a sample of the visibilities for plotting
n = 100000
x = np.random.choice(len(uval.data.flatten()), n)

In [12]:
# Sorting the sample indicies makes the following much faster
x.sort()

In [13]:
# Flatten from 82144 x 4096 to 1 dimension then take the sample
u = uval.data.flatten()[x]
v = vval.data.flatten()[x]

In [14]:
u = u.compute().squeeze()
v = v.compute().squeeze()

In [17]:
# Bokeh plot for 2D scatter plot 
#      The color represents the order in the observation where later 
#      visibilities have lighter color to show georotational interferometery

TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

p = figure(tools=TOOLS)

colors = [
    "#%02x%02x%02x" % (int(r), int(g), 150) for r, g in zip(50+200*x/np.max(x), 30+200*x/np.max(x))
]

p.scatter(u, v, fill_color=colors, fill_alpha=0.7,
          line_color=None)

GlyphRenderer(id='1055', ...)

In [18]:
show(p)

**caption**: color scale represents position in the dataset regarding time.  So the shading shows the progression of earth-rotation aperture sytheses as the observation progresses in time.

---

## Apply Statistics to Bins

In [4]:
dd_ubins = da.from_array(ds_ind.U_bins)
dd_vbins = da.from_array(ds_ind.V_bins)
dd_vals = da.from_array(ds_ind.DATA[:,0])

In [5]:
dd_vals 

dask.array<array, shape=(336461824,), dtype=complex64, chunksize=(10000000,), chunktype=xarray.DataArray>

In [6]:
dd_bins = da.stack([dd_ubins, dd_vbins]).T
dd_bins

dask.array<transpose, shape=(336461824, 2), dtype=int64, chunksize=(10000000, 1), chunktype=numpy.ndarray>

In [7]:
chunk_size = 10**7
dd_bins = dd_bins.rechunk([chunk_size, 2])
dd_vals = dd_vals.rechunk([chunk_size, 1])

In [8]:
bin_partitions = dd_bins.to_delayed()
val_partitions = dd_vals.to_delayed()

In [114]:
value_group_chunks = [dask.delayed(groupby_apply.group_bin_values_wrap)(part[0][0], part[1]) for part in zip(bin_partitions, val_partitions)]   

In [115]:
value_groups_ = dask.delayed(groupby_apply.combine_group_values)(grouped_values)

In [116]:
value_groups = value_groups_.compute()

In [113]:
len(bin_values)

30116

In [30]:
result = grouped_values[0].compute()

In [101]:
len(result), len(result[0])

(494, 243)

In [36]:
funtion_result = dask.delayed(groupby_apply.apply_to_groups)(result, np.median)

---

In [85]:
ubin_number = 201
selbins = []
for i, r_ in enumerate(result[ubin_number]):
    if len(r_) > 0:
        print(f'({ubin_number}, {i}):\t {len(r_)}')
        selbins.append((ubin_number, i))

(201, 97):	 968
(201, 98):	 2321
(201, 99):	 5355
(201, 100):	 4366
(201, 102):	 12
(201, 103):	 3473
(201, 104):	 2623
(201, 105):	 1162
(201, 106):	 1692
(201, 111):	 2377
(201, 199):	 10
(201, 200):	 1476
(201, 201):	 2149
(201, 202):	 368
(201, 222):	 7
(201, 223):	 562
(201, 224):	 1320
(201, 225):	 1165
(201, 226):	 335


---

## Get Bin Index Groups

In [92]:
index_group_chunks = [dask.delayed(groupby_apply.groupby_nd_wrap)(part[0], init_index=int(chunk_size*i)) for i, part in enumerate(bin_partitions)]

In [94]:
index_groups_ = dask.delayed(groupby_apply.combine_ind_chunks)(index_group_chunks)

In [95]:
index_groups = index_groups_.compute() 

---

### Plot Bin Variance 

Now use **bokeh**

In [48]:
selbin = result[247][158]

In [ ]:
selbin = result[]

In [49]:
len(selbins)

696

In [59]:
nbins = 100

In [60]:
hist, edges = np.histogram(selbin, density=True, bins=nbins)

In [58]:
p = figure(plot_height=400, plot_width=600, title="Bin Amplitude Distribution", x_axis_label="Amplitude", y_axis_label="N")

In [62]:
p.quad(bottom=0, top=hist, left=edges[:-1], right=edges[1:], fill_color='navy', alpha=0.7)

GlyphRenderer(id='1039', ...)

In [63]:
show(p)

### Gridplot of Bin Distribution 

In [68]:
from bokeh.layouts import gridplot 

In [78]:
def make_hist_plot(binname, bin_values, nbins):
    hist, edges = np.histogram(bin_values, density=True, bins=nbins)
    
    p = figure(title=f'{binname}', tools='')
    p.quad(bottom=0, top=hist, left=edges[:-1], right=edges[1:], fill_color='navy', alpha=0.7)
    return p

In [71]:
selbins = [(200, 96), (200, 97), (200, 98), (200, 99), (200, 102), (200, 103), (200, 104), (200, 105), (200, 110), (200, 111), (200, 155), (200, 201), (200, 202), (200, 225), (200, 226), (200, 227)]

In [86]:
len(selbins)

19

In [87]:
nbins = 100
plots = []
for sb in selbins:
    print(len(result[sb[0]][sb[1]]))
    plots.append( make_hist_plot(sb, result[sb[0]][sb[1]], nbins) )

968
2321
5355
4366
12
3473
2623
1162
1692
2377
10
1476
2149
368
7
562
1320
1165
335


In [81]:
len(plots)

16

In [83]:
show(gridplot(plots, ncols=4, plot_width=200, plot_height=200, toolbar_location=None))

In [88]:
show(gridplot(plots, ncols=5, plot_width=200, plot_height=200, toolbar_location=None))

---


### Inspect One Bin 

In [108]:
dd_uv = da.from_array(ds_ind.UV)

In [96]:
u_ind = 201
v_ind = 99

In [97]:
bin_group = index_groups[u_ind][v_ind]

In [98]:
bin_values = dd_vals[bin_group]

In [109]:
bin_values = dd_uv[bin_group]

In [110]:
len(bin_values)

30116

In [112]:
bin_values[:10].compute()

array([[ -5347.38442541, -11703.82680565],
       [ -5348.22270483, -11705.66154884],
       [ -5349.06098425, -11707.49629203],
       [ -5349.89926367, -11709.33103522],
       [ -5350.73754309, -11711.16577841],
       [ -5351.57582251, -11713.0005216 ],
       [ -5352.41410193, -11714.83526479],
       [ -5353.25238135, -11716.67000798],
       [ -5354.09066077, -11718.50475117],
       [ -5354.92894019, -11720.33949436]])

In [99]:
len(bin_values)

30116

In [100]:
len(result[u_ind][v_ind])

5355

In [ ]:
result[sb[0]][sb[1]]
bin_values

In [ ]:
p = figure(tools=TOOLS, x_axis_label="Amplitude", y_axis_label="Position")

x = result[sb[0]][sb[1]]
y = bin_values

p.scatter(x, y, radius=2., fill_color="green", fill_alpha=0.6, line_color=None)

---

In [27]:
from importlib import reload

In [ ]:


result_ = function_result.compute()

In [28]:
reload(groupby_apply)

<module 'groupby_apply' from '/users/jbochenek/rfi_flagging/groupby_apply.py'>

In [124]:
val_partitions[0]

Delayed(('array-1e80d59857948f45f9e9e17e117f63f1', 0))

In [126]:
dask.delayed(group_bin_test)(bin_partitions[0][0], val_partitions[0]).compute()

0 237 237 (4513.9224+0j)
1 139 139 (84.310646+0j)


In [ ]:
groupby_apply.group_bin_values_wrap()

In [ ]:
# Test on a subset of partitions 
grouped_values = [dask.delayed(group_bin_values_wrap)(part[0]) for i, part in enumerate(partitions)]